In [1]:
# |default_exp parsing.ms_office.marker

In [2]:
# | hide
from nbdev.showdoc import *
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

## Install dependencies

In [3]:
# | export
import sys
import os
import time
from hashlib import sha256
from pathlib import Path
from pprint import pprint, pformat

from odfdo import Document
from PIL import Image
import pandas as pd
# import ezodf

In [4]:
#| export
from dotenv import load_dotenv
load_dotenv()
OPENROUTER_API_KEY= os.getenv('OPENROUTER_API_KEY')
# print(OPENROUTER_API_KEY)
OPENROUTER_API_URL = os.getenv('OPENROUTER_API_URL')
# print(OPENROUTER_API_URL)
# OPENROUTER_MODEL = os.getenv('OPENROUTER_MODEL')
OPENROUTER_MODEL = 'Google:gemini-2.5-pro-exp-03-25'
# print(OPENROUTER_MODEL)

True

In [5]:
def convert_office_to_pdf(root_folder):
    """
    Recursively convert all .pptx, .ppt, .docx, and .doc files under
    root_folder (and subfolders) to PDF files in the same folder as the
    original. For .xls and .xlsx files, convert to markdown (.md) files
    in the same folder. Requires LibreOffice (soffice) installed for Office
    document conversion. Requires pandas for Excel to markdown.
    """
    import subprocess
    from pathlib import Path
    import pandas as pd

    office_exts = {'.pptx', '.ppt', '.docx', '.doc'}
    excel_exts = {'.xls', '.xlsx'}
    root = Path(root_folder)
    for file in root.rglob('*'):
        if file.suffix.lower() in office_exts:
            pdf_path = file.with_suffix('.pdf')
            if pdf_path.exists():
                continue  # Skip if PDF already exists
            try:
                # Use soffice for conversion
                subprocess.run([
                    'soffice', '--headless', '--convert-to', 'pdf',
                    '--outdir', str(file.parent), str(file)
                ], check=True)
                print(f"Converted: {file} -> {pdf_path}")
            except subprocess.CalledProcessError as e:
                print(f"Failed to convert {file}: {e}")
        elif file.suffix.lower() in excel_exts:
            md_path = file.with_suffix('.md')
            if md_path.exists():
                continue
            try:
                excel = pd.read_excel(file, sheet_name=None)
                with open(md_path, 'w', encoding='utf-8') as f:
                    for sheet, df in excel.items():
                        f.write(f'# Sheet: {sheet}\n\n')
                        f.write(df.to_markdown(index=False))
                        f.write('\n\n')
                print(f"Converted: {file} -> {md_path}")
            except Exception as e:
                print(f"Failed to convert {file} to markdown: {e}")
        else:
                print(f"Failed to convert {file}")


In [6]:

# Example usage:
convert_office_to_pdf('/v/data/新型机器人智能问答系统数据源')
# convert_office_to_pdf('../res/conversion_test')


In [6]:
import pandas as pd
from pathlib import Path

def convert_excel_to_markdown(file_path):
    """
    Convert an Excel file to a markdown file.
    """
    try:
        excel = pd.read_excel(file_path, sheet_name=None)
        md_path = file_path.with_suffix('.md')
        with open(md_path, 'w', encoding='utf-8') as f:
            for sheet, df in excel.items():
                f.write(f'# Sheet: {sheet}\n\n')
                f.write(df.to_markdown(index=True))
                f.write('\n\n')
        print(f"Converted: {file_path} -> {md_path}")
    except Exception as e:
        print(f"Failed to convert {file_path} to markdown: {e}")

# convert_excel_to_markdown(Path('../res/conversion_test/test.xlsx'))  # Example usage

In [8]:
# print(f'--gemini_api_key={GEMINI_KEY}')

# print(f'--openai_api_key={OPENROUTER_API_KEY}')
# print(f'--openai_model={OPENROUTER_MODEL}')
# print(f'--openai_base_url={OPENROUTER_API_URL}')

In [7]:
def convert_pdf_to_markdown_with_marker(root_folder):
    """
    Recursively convert all .pdf files under root_folder (and subfolders)
    to markdown (.md) files in the same folder using marker-pdf's 'marker' command
    with 4 workers.
    """
    import subprocess
    from pathlib import Path
    # import os

    root = Path(root_folder)
    md_root = root / '.md'
    for file in root.rglob('*.pdf'):
        md_path = file.with_suffix('.md')
        md_path = md_root / md_path.relative_to(root) # store the result md files into a separate path of 'root/.md'
        if md_path.exists():
            continue  # Skip if markdown already exists
        try:
            # subprocess.run([
            #     'marker_single', str(file), '--output_dir', file.parent, 
            #     '--use_llm',
            #     f'--gemini_api_key={GEMINI_KEY}',
            # ], check=True)
            #subprocess.run([
            #    'marker_single', str(file), '--output_dir', file.parent, 
            #    '--llm_service=marker.services.ollama.OllamaService',
            #    '--ollama_base_url=http://172.27.74.15:11434',
            #    '--ollama_model=gemma3:27b',
            #], check=True)
            subprocess.run([
                'marker_single', str(file), '--output_dir', md_path.parent, 
                '--llm_service=marker.services.openai.OpenAIService',
                f'--openai_api_key={OPENROUTER_API_KEY}',
                f'--openai_model={OPENROUTER_MODEL}',
                f'--openai_base_url={OPENROUTER_API_URL}',
            ], check=True)
            print(f"Converted: {file} -> {md_path}")
        except subprocess.CalledProcessError as e:
            print(f"Failed to convert {file}: {e}")

In [11]:
# convert_pdf_to_markdown_with_marker('/v/data/新型机器人智能问答系统数据源-gemini')
convert_pdf_to_markdown_with_marker('../res/warning')
# convert_pdf_to_markdown_with_marker('../res/conversion_test')

Recognizing tables: 100%|██████████| 22/22 [00:10<00:00,  2.07it/s]
2025-06-11 17:24:25,281 [INFO] marker: Saved markdown to ../res/warning/.md/报警码
2025-06-11 17:24:25,281 [INFO] marker: Total time: 182.80693531036377


Converted: ../res/warning/报警码.pdf -> ../res/warning/.md/报警码.md


In [8]:
def convert_excel_to_markdown_with_marker(root_folder):
    """
    Recursively convert all .excel files under root_folder (and subfolders)
    to markdown (.md) files in the same folder using marker-pdf's 'marker' command
    with 4 workers.
    """
    import subprocess
    from pathlib import Path
    # import os

    root = Path(root_folder)
    md_root = root / '.md'
    for file in root.rglob('*.xlsx'):
        md_path = file.with_suffix('.md')
        md_path = md_root / md_path.relative_to(root) # store the result md files into a separate path of 'root/.md'
        if md_path.exists():
            continue  # Skip if markdown already exists
        try:
            # subprocess.run([
            #     'marker_single', str(file), '--output_dir', file.parent, 
            #     '--use_llm',
            #     f'--gemini_api_key={GEMINI_KEY}',
            # ], check=True)
            #subprocess.run([
            #    'marker_single', str(file), '--output_dir', file.parent, 
            #    '--llm_service=marker.services.ollama.OllamaService',
            #    '--ollama_base_url=http://172.27.74.15:11434',
            #    '--ollama_model=gemma3:27b',
            #], check=True)
            subprocess.run([
                'marker_single', str(file), '--output_dir', md_path.parent, 
                '--llm_service=marker.services.openai.OpenAIService',
                f'--openai_api_key={OPENROUTER_API_KEY}',
                f'--openai_model={OPENROUTER_MODEL}',
                f'--openai_base_url={OPENROUTER_API_URL}',
            ], check=True)
            print(f"Converted: {file} -> {md_path}")
        except subprocess.CalledProcessError as e:
            print(f"Failed to convert {file}: {e}")

In [9]:
convert_excel_to_markdown_with_marker('../res/test_cases')

Running OCR Error Detection: 100%|██████████| 2/2 [00:00<00:00, 80.74it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing tables: 100%|██████████| 2/2 [00:00<00:00,  2.71it/s]
2025-06-12 10:08:46,387 [INFO] marker: Saved markdown to ../res/test_cases/.md/AI+项目测试用例_raw
2025-06-12 10:08:46,387 [INFO] marker: Total time: 6.024226188659668


Converted: ../res/test_cases/AI+项目测试用例_raw.xlsx -> ../res/test_cases/.md/AI+项目测试用例_raw.md


In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()

<div>
<link rel="stylesheet" href="https://gradio.s3-us-west-2.amazonaws.com/2.6.5/static/bundle.css">
<div id="target"></div>
<script src="https://gradio.s3-us-west-2.amazonaws.com/2.6.5/static/bundle.js"></script>
<script>
launchGradioFromSpaces("abidlabs/question-answering", "#target")
</script>
</div>